In [13]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import mean_squared_error, accuracy_score
import json
from tqdm import tqdm
import numpy as np
import os

In [30]:
# Load data
with open('data/poetry_data_train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


data[0]

{'url': 'https://www.chitalnya.ru/work/3180020/',
 'rating': '29',
 'views': 33,
 'output_text': 'Люблю ли осень? Ты спроси у ели -\nОна в колючках и не ждёт тепла...\nЕй хрен один: повсюду лишь метели,\nКапель ли хнычет ночью со стекла.\n\nБагряный лист, ли, кружится прощаясь,\nЛи фонаря торчащего кадык,\nНа низком небе пятнами касаясь\nК луне общаясь, притулился встык.\n\nЛюблю ли осень? Ты спроси у ветра.\nС ним всё равно не сладить в парусах.\nПо осени ли, по весне для шторма,\nСпускают их, на реи намотав.\n\nПожухлых трав не кошенные дали,\nЧьих жалких тел охапками нажнут.\nСпроси у них, о том они мечтали\nВесной, тогда!? Теперь их просто жгут.\n\nПо осени, когда седеют чувства,\nЗастыли ветви рук на деревах -\nМне хочется упасть на дно колодца,\nЧто бы не видеть слёзы на глазах.',
 'genre': 'лирика'}

In [34]:
poems = [entry['output_text'] for entry in data if float(entry['rating']) > 0 and entry['views'] >= 50]
ratings = [float(entry['rating']) for entry in data if float(entry['rating']) > 0 and entry['views'] >= 50]

In [35]:
import pandas as pd

ratings = pd.qcut(ratings, 10, labels=False)  # Разделяет рейтинги на 10 категорий
ratings = ratings.tolist()

np.unique(ratings)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
class PoemRatingDataset(Dataset):
    def __init__(self, poems, ratings, tokenizer):
        self.poems = poems
        self.ratings = ratings
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.poems)

    def __getitem__(self, idx):
        poem = self.poems[idx]
        rating = self.ratings[idx]
        encoding = self.tokenizer(
            poem,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(rating, dtype=torch.long)  # Изменение на целочисленную метку
        return item

In [ ]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
dataset = PoemRatingDataset(poems, ratings, tokenizer)

# Train-validation split
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [15]:
from transformers import BertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=10).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps=1e-8)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
num_epochs = 4

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Training loss: {total_train_loss / len(train_dataloader)}")

    # Валидация
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    print(f"Validation loss: {total_val_loss / len(val_dataloader)}")

100%|██████████| 2237/2237 [12:23<00:00,  3.01it/s]


Epoch 1/4, Training loss: 2.256960899796825
Validation loss: 2.260660559444104


100%|██████████| 2237/2237 [12:23<00:00,  3.01it/s]


Epoch 2/4, Training loss: 2.258260872782902
Validation loss: 2.255840838965723


100%|██████████| 2237/2237 [12:23<00:00,  3.01it/s]


Epoch 3/4, Training loss: 2.2579921418447073
Validation loss: 2.2549050318992743


100%|██████████| 2237/2237 [12:23<00:00,  3.01it/s]


Epoch 4/4, Training loss: 2.256723910450243
Validation loss: 2.25733257956424


In [17]:
torch.save(model.state_dict(), f'bert_classification/classification_poem_rating_model_filtered.pth')
tokenizer.save_pretrained(f'bert_classification/tokenizer/')

('bert_classification/tokenizer/tokenizer_config.json',
 'bert_classification/tokenizer/special_tokens_map.json',
 'bert_classification/tokenizer/vocab.txt',
 'bert_classification/tokenizer/added_tokens.json')

### Расчет NDCG на тестовой выборке

In [38]:
# Load data
with open('data/poetry_data_test.json', 'r', encoding='utf-8') as file:
    test_data = json.load(file)


test_data[0]

{'url': 'https://www.chitalnya.ru/work/2999707/',
 'rating': '0',
 'views': 15,
 'output_text': 'Нынче головы четыре у дракона,\nПожиратель президентов он мастак.\nНа него управы нет, и нет закона,\nВ споре с ним же попадёт любой впросак.\n\nУ него в кармане сотни триллионов.\nДля него богач - обычный нищеброд.\nНету больше никаких на свете тронов,\nНи во что не ставит он любой народ.\n\nОн прикажет слугам - выпилить любого\nС интернета, те его прогонят прочь,\nИ никто не пикнет, не промолвит слова,\nНекому бедняге на земле помочь.\n\nИ всё потому, что присягнули змею,\nОтступив от Бога, скорбен наш удел.\nСлужат люди молча мерзкому пигмею,\nПоголовно, будто вовсе оборзев.\n\nНынче люди лживы, врут напраполую.\nОн сегодня баба, завтра же мужик.\nГрабят, убивают, даже мать родную\nМогут погубить, коль сам дракон велит.\n\nВремена драконьи - времена инферно.\nКовид пандемия - следствие его,\nОбезумевши, спешит планету ввергнуть\nВ бездну - так уже желает бошинство.',
 'genre': 'лирика'}

In [10]:
from transformers import BertForSequenceClassification

path = 'bert_classification/classification_poem_rating_model_filtered.pth'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=10)
model.load_state_dict(torch.load(path, weights_only=True))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 714.29 MB. The target location /root/.cache/huggingface/hub/models--DeepPavlov--rubert-base-cased/blobs only has 0.00 MB free disk space.
  warnings.warn(


In [43]:
poems = [entry['output_text'] for entry in test_data]
ratings = [float(entry['rating']) for entry in test_data]

In [42]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
test_dataset = PoemRatingDataset(poems, ratings, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [ ]:
predictions = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        predicted_ratings = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(predicted_ratings)

# Теперь у вас есть список предсказанных рейтингов
predictions = np.array(predictions)
true_ratings = np.array(ratings)


In [ ]:
import numpy as np

def calculate_dcg(scores, k=10):
    """Calculate DCG given a list of relevance scores, limited to top k positions."""
    gains = scores / np.log2(np.arange(2, scores.size + 2))
    return np.sum(gains[:k])

def ndcg_score_large_dataset(y_true, y_pred_classes, k=10):
    ndcg_scores = []
    for true_ratings, pred_classes in zip(y_true, y_pred_classes):
        sorted_pred_indices = np.argsort(pred_classes)[::-1]
        sorted_true_indices = np.argsort(true_ratings)[::-1]

        pred_ordered_true_ratings = true_ratings[sorted_pred_indices]
        
        dcg = calculate_dcg(pred_ordered_true_ratings, k)
        idcg = calculate_dcg(true_ratings[sorted_true_indices], k)

        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcg_scores.append(ndcg)

    return np.mean(ndcg_scores)


ndcg = ndcg_score_large_dataset(true_ratings, predictions, k=10)
print(f"Normalized Discounted Cumulative Gain (NDCG): {ndcg}")